# 2D Variable Aggregation and Extraction

This notebook provides a tool to extract specific 2D variables from PALM (Potsdam Atmospheric Large-Eddy Simulation Model) NetCDF output files and optionally perform temporal aggregation. The aggregated data is then saved into new NetCDF files, facilitating further analysis and visualization without needing to process large raw simulation outputs repeatedly.


## 1. Import Dependencies

This section imports all necessary libraries for numerical operations, NetCDF file handling, interactive widgets for user input, and operating system interactions.

In [3]:
import os

import numpy as np
import netCDF4 as nc
from netCDF4 import Dataset

from IPython.display import display
import ipywidgets as widgets

from utils import palm_variables

## 2. Load Simulation Data
This section defines the absolute paths to the PALM simulation output files (xy-averaged and static driver) and loads them into NetCDF dataset objects. These datasets provide access to the raw simulation variables.

In [ ]:
# Absolute URLs (paths) of simulation output files and the static driver file
file_xy_1 = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Baseline-48hr/OUTPUT/konstanz_4096x4096_v9_Baseline_av_xy_N03.000.nc"
file_xy_2 = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Scenario_1-48hr/OUTPUT/konstanz_4096x4096_v9_Scenario_1_av_xy_N03.000.nc"
file_static = r"./Data/_simulation_outputs_3/konstanz_4096x4096_v9_Scenario_1-48hr/INPUT/konstanz_4096x4096_v9_Scenario_1_static_N03"

# Read NetCDF files into Dataset objects in read mode ('r')
dataset_1 = nc.Dataset(file_xy_1, mode='r')
dataset_2 = nc.Dataset(file_xy_2, mode='r')
dataset_3 = nc.Dataset(file_static, mode='r')

# Store datasets in a list for easier iteration
dataset_list = [dataset_1, dataset_2]
file_xy_list = [file_xy_1, file_xy_2]

## 3. Variable Selection
This section allows the user to interactively select a 2D variable from the loaded NetCDF datasets. A dropdown widget is used for selection, and the corresponding variable's description and unit are displayed for user convenience, fetched from the palm_variables module.

In [6]:
# Extract variable names from the first dataset that have more than 2 dimensions (typically 3D or 4D variables)
# These usually represent spatial or spatio-temporal data (e.g., time, z, y, x or time, y, x)
var_names_palm = [var for var in dataset_1.variables if dataset_1.variables[var].ndim > 2]

# Initialize the test_variable with the first variable in the list
test_variable = var_names_palm[0]

# Create a dropdown widget for selecting the test variable
drop_down = widgets.Dropdown(
    options=var_names_palm,         # Populate dropdown with variable names
    value=var_names_palm[0],        # Set initial selected value
    description='Select test variable:' # Label for the dropdown
)

# Define a handler function to update the global test_variable when a new option is selected
def dropdown_handler(change):
    global test_variable
    test_variable = change.new # Update the global variable with the new selection
    print(f"Selected variable: {test_variable}") # Print the newly selected variable

# Attach the dropdown_handler function to observe changes in the 'value' property of the dropdown
drop_down.observe(dropdown_handler, names='value')

# Display the dropdown widget to the user
display(drop_down)

Dropdown(description='Select test variable:', options=('ta_2m*_xy', 'tsurf*_xy', 'wspeed_10m*_xy', 'bio_pet*_x…

## 4. Aggregation Logic
This section defines the core logic for temporal aggregation. The get_aggregate_time_list function creates a list of time step ranges based on a specified aggregation window. This allows for calculating moving averages over the simulation data.

In [7]:
def get_aggregate_time_list(total_time_steps, aggregate_time_steps):
    """
    Generates a list of time step indices for temporal aggregation.
    Each sub-list represents a window for calculating a moving average.

    Args:
        total_time_steps (int): The total number of time steps in the simulation.
        aggregate_time_steps (int): The number of time steps to include in each aggregation window.

    Returns:
        list: A list of lists, where each inner list contains time indices
              for a specific aggregation window.
    """
    time_lists = []
    
    for i in range(total_time_steps):
        # Case 1: No aggregation (single time step per window)
        if aggregate_time_steps <= 1:
            time_list = [i]
        # Case 2: Aggregation with a window (moving average)
        else:
            # Calculate the half-window size
            half_window = aggregate_time_steps // 2
            # Determine the start and end indices of the window based on even/odd aggregation steps
            if aggregate_time_steps % 2 == 0:
                # For even `aggregate_time_steps`, the window is centered such that `i` is towards the end of the first half.
                # Example: for aggregate_time_steps=6, half_window=3. Window for `i` is [i-3, i-2, i-1, i, i+1, i+2]
                time_list = [j for j in range(i - half_window, i + half_window)]
            else:
                # For odd `aggregate_time_steps`, the window is perfectly centered around `i`.
                # Example: for aggregate_time_steps=5, half_window=2. Window for `i` is [i-2, i-1, i, i+1, i+2]
                time_list = [j for j in range(i - half_window, i + half_window + 1)]
        
        # Filter out time indices that are outside the total simulation time steps
        valid_time_list = [j for j in time_list if 0 <= j < total_time_steps]
        time_lists.append(valid_time_list)
    
    return time_lists

In [9]:
# Extract filename (without extension) for dataset_1
filename_xy_1 = os.path.basename(file_xy_1).split('.')[0]
# Extract filename (without extension) for dataset_2 (corrected from dataset_1 to dataset_2)
filename_xy_2 = os.path.basename(file_xy_2).split('.')[0]

# Load the actual variable data for the selected 'test_variable' from both datasets
variable_data_1 = dataset_1[test_variable]
variable_data_2 = dataset_2[test_variable]

# Get the shape of the variable data to determine total time steps
variable_data_shape = np.shape(dataset_1[test_variable])
total_time_steps = variable_data_shape[0]

# Define the aggregation window size (e.g., 1 for no aggregation, 6 for 1-hour average)
# This can be changed to perform different levels of temporal aggregation
aggregate_time_steps = 1 # Default to no aggregation (individual time steps)

# Generate the list of time step ranges for aggregation based on the defined window
time_lists = get_aggregate_time_list(total_time_steps, aggregate_time_steps)

## 5. Save Aggregated Data
This section iterates through the loaded datasets, aggregates the selected variable data based on the defined time windows, and saves the aggregated results into new NetCDF files. Each aggregated file will contain the averaged values for the chosen variable, reflecting the specified aggregation period.

In [ ]:
# Iterate through each dataset in the dataset_list (e.g., Baseline and Scenario 1)
for i, dataset_n in enumerate(dataset_list):
    # Extract the variable data for the currently selected test_variable from the current dataset
    variable_data = dataset_n[test_variable]
    
    # Get the total number of time steps for the current variable data
    total_time_steps = np.shape(variable_data)[0]
    
    # Regenerate time_lists in case `total_time_steps` is different for different datasets,
    # or if `aggregate_time_steps` has been changed interactively.
    time_lists = get_aggregate_time_list(total_time_steps, aggregate_time_steps)
    
    # Initialize a list to store the aggregated data for all time steps
    variable_data_agg = []
    
    # Loop through each generated time window (time_list)
    for j, time_list in enumerate(time_lists):
        values_temp_list = []
        
        # For each time window, extract the 2D variable data slices
        for time_index in time_list:
            # Extract data at the current time_index, assuming it's a 4D variable (time, z, y, x)
            # and we're interested in the first z-layer (index 0)
            values_temp = variable_data[time_index, 0, :, :]
            values_temp_list.append(values_temp)
            
        # Compute the mean of the collected data slices along the time axis (axis=0)
        variable_data_agg.append(np.mean(values_temp_list, axis=0))
        
    # Determine the base filename for the output NetCDF file
    # Uses the original filename (e.g., 'konstanz_4096x4096_v9_Baseline_av_xy_N03')
    source_filename = os.path.basename(file_xy_list[i]).split('.')[0]
    
    # Construct the output filename, replacing '*' in test_variable if present
    # Appends the variable name to the source filename for clarity
    output_filename = f"{source_filename}_{test_variable.replace('*','')}"
    
    # Create the output directory if it does not exist
    output_dir = f"./output/_01_agg_{aggregate_time_steps}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Save the aggregated data to a new NetCDF file
    with Dataset(os.path.join(output_dir, f"{output_filename}.nc"), "w", format='NETCDF4_CLASSIC') as dataset:
        # Define dimensions for the new NetCDF file
        num_time = len(variable_data_agg)
        num_z_index = 1 # We've flattened the z-dimension by selecting index 0
        num_rows = variable_data_agg[0].shape[0] # Y-dimension
        num_cols = variable_data_agg[0].shape[1] # X-dimension
        
        dataset.createDimension('time', num_time)
        dataset.createDimension('z', num_z_index)
        dataset.createDimension('x', num_cols) # Corrected to match convention (x is usually columns)
        dataset.createDimension('y', num_rows) # Corrected to match convention (y is usually rows)
        
        # Create the variable in the new NetCDF file
        # The dimensions are (time, z, y, x)
        data_var = dataset.createVariable(f'{test_variable}', np.float32, ('time', 'z', 'y', 'x'), fill_value=-9999.0)
        
        # Fill the newly created variable with the aggregated data
        for k, array in enumerate(variable_data_agg):
            data_var[k, 0, :, :] = array # Assign each aggregated 2D array to its time step and z-layer

    print(f"Aggregated data for {test_variable} saved to {output_dir}/{output_filename}.nc")

Aggregated data for ta_2m*_xy saved to ./output/001_agg_1/konstanz_4096x4096_v9_Baseline_av_xy_N03_ta_2m_xy.nc
Aggregated data for ta_2m*_xy saved to ./output/001_agg_1/konstanz_4096x4096_v9_Scenario_1_av_xy_N03_ta_2m_xy.nc
